In [7]:
import pandas as pd
import glob
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [8]:
filename = "../../results_aggr/app_bootstrap_vm.csv"
outfilePrefix = "../../results_aggr/app_bootstrap_vm_plots"
df_values = pd.read_csv(filename, names=["commit", "type", "CI_min", "Median", "CI_max", "change"], skiprows=1)
df_values.sort_values(by=["commit"], inplace=True)

In [9]:
for type in df_values.type.unique():
    print(f"Plotting type {type}")

    measurements = df_values.loc[(df_values['type'].str.startswith(type, na=False))]


    #Find % set y-limits
    min = -5
    max = 5
    for minSmall in measurements.CI_min.unique():
        if minSmall < min:
            min = minSmall

    for maxSmall in measurements.CI_max.unique():
        if maxSmall > max:
            max = maxSmall


    sns.set(style="whitegrid")
    axes = plt.axes()
    axes.set_ylim([min, max])

    #Plot CIs
    plot = sns.lineplot(data=measurements, x="commit", y="CI_min", color="royalblue")
    plot = sns.lineplot(data=measurements, x="commit", y="CI_max", color="royalblue")
    x_commits = measurements.commit
    plot.fill_between(x=x_commits, y1=measurements.CI_min, y2=measurements.CI_max, facecolor = 'lightblue')

    #Plot Median
    plot = sns.lineplot(data=measurements, x="commit", y="Median", color="red")

    #PLot Up&Down Changes
    ups = measurements.loc[(measurements['change'].str.startswith("Up", na=False))]
    plot = sns.regplot(data=ups,x="commit", y="Median", marker="^", fit_reg=False, scatter_kws={"color":"black","alpha":1,"s":300})
    downs = measurements.loc[(measurements['change'].str.startswith("Down", na=False))]
    plot = sns.regplot(data=downs,x="commit", y="Median", marker="v", fit_reg=False, scatter_kws={"color":"black","alpha":1,"s":300})

    plot.set(ylim=(min, max))
    plt.savefig(f'{outfilePrefix}/{type}.png')
    plt.clf()


Plotting type inserts
Plotting type simple queries
Plotting type group-by queries


<Figure size 432x288 with 0 Axes>